In [1]:
import pandas as pd
from pandas import DataFrame, Series

from src.lib import version_extractor, name_extractor

pd.set_option('display.max_columns', 100)

In [2]:
pypi_data = pd.read_json('../../data/repology/pypicache.json')
pypi_data

,info,last_serial,releases,urls,vulnerabilities
0,"{'author': 'Dmitry Berezovsky', 'author_email'...",8973353,"{'0.0.8': [{'comment_text': '', 'digests': {'m...","[{'comment_text': '', 'digests': {'md5': '360c...",[]
1,"{'author': 'Abraham', 'author_email': 'abraham...",9481777,"{'1.0.0': [{'comment_text': '', 'digests': {'m...","[{'comment_text': '', 'digests': {'md5': 'dfef...",[]
2,"{'author': 'Sergei Minaev', 'author_email': 'i...",9764135,"{'0.5': [{'comment_text': '', 'digests': {'md5...","[{'comment_text': '', 'digests': {'md5': 'b612...",[]
3,{'author': 'Wolfgang Schnerring <wosc@wosc.de>...,11184293,"{'1.0.4': [{'comment_text': '', 'digests': {'m...","[{'comment_text': '', 'digests': {'md5': '55ed...",[]
4,"{'author': 'Thea Barnes', 'author_email': 'the...",11644911,"{'0.0.1': [{'comment_text': '', 'digests': {'m...","[{'comment_text': '', 'digests': {'md5': '3613...",[]
...,...,...,...,...,...
169616,"{'author': 'Amazon Web Services', 'author_emai...",13647017,"{'1.154.0': [{'comment_text': '', 'digests': {...","[{'comment_text': '', 'digests': {'md5': 'ae15...",[]
169617,"{'author': 'Amazon Web Services', 'author_emai...",13647009,"{'1.154.0': [{'comment_text': '', 'digests': {...","[{'comment_text': '', 'digests': {'md5': '9bc9...",[]
169618,"{'author': '', 'author_email': '', 'bugtrack_u...",13625952,"{'0.0.23': [{'comment_text': '', 'digests': {'...","[{'comment_text': '', 'digests': {'md5': '0d0a...",[]
169619,"{'author': 'Amazon Web Services', 'author_emai...",13647157,"{'1.154.0': [{'comment_text': '', 'digests': {...","[{'comment_text': '', 'digests': {'md5': 'dde6...",[]


### Converting Info JSON to a DataFrame

In [3]:
# pypi_data['info'].map(lambda x: x['author'])
# df = pypi_data['info'].apply(pd.Series) --- Slow in general. For the given case, just as fast as json_normalize (without a set max level) as that attempts to unnest the entire json object. ~ 30 seconds
# df = pd.json_normalize(pypi_data['info']) --- Slow due to previously mentioned unnesting. ~ 30 seconds
# df = pd.json_normalize(pypi_data['info'], max_level=0) --- Faster. ~ 5 seconds
# df = pd.DataFrame(pypi_data['info'].values.tolist()) --- Fastest. ~ 0.5 seconds

info_df = pd.DataFrame(pypi_data['info'].values.tolist())
info_df = info_df[['name', 'version', 'requires_dist', 'author']]
# Rename headers to make it more readable
info_df.rename(columns={'requires_dist': 'dependency', }, inplace=True)

In [4]:
sorted_df: DataFrame = info_df.sort_values(by=['name', 'version'], ascending=[True, False], ignore_index=True)
sorted_df

,name,version,dependency,author
0,024travis-test024,0.1.0,None,Travis Torline
1,0x-order-utils,4.0.1,"[0x-contract-addresses, 0x-contract-artifacts,...",F. Eugene Aumson
2,0x-python,1.0.16,[requests],Skeetzo
3,0x20bf,0.0.1,"[PyYAML (==5.4.1), aiohttp (==3.7.4.post0), gn...",randymcmillan
4,0x2nac0nda,0.1,None,Abdalla Abdelrhman
...,...,...,...,...
169616,zzy-AxCat,0.1,None,zzy
169617,zzz001,0.0.4,[PyYAML],Jasper Jin
169618,zzzPyPiTest,0.0.2,None,zhaozizhe
169619,zzzPyPiTest2,0.0.2,None,zhaozizhe


In [5]:
def extract_date_from_nested_releases_json(releases_json):
    if isinstance(releases_json, dict):
        latest_release = [*releases_json.values()][0]
        if latest_release:
            return latest_release[0]['upload_time']
    else:
        return None

In [6]:
upload_time_series: Series = pypi_data['releases'].map(extract_date_from_nested_releases_json)

In [7]:
sorted_df.insert(loc=2, column='upload_time', value=upload_time_series)
sorted_df

,name,version,upload_time,dependency,author
0,024travis-test024,0.1.0,2020-12-23T22:08:32,None,Travis Torline
1,0x-order-utils,4.0.1,2021-02-21T15:59:48,"[0x-contract-addresses, 0x-contract-artifacts,...",F. Eugene Aumson
2,0x-python,1.0.16,2021-03-14T03:06:40,[requests],Skeetzo
3,0x20bf,0.0.1,2021-08-15T16:26:14,"[PyYAML (==5.4.1), aiohttp (==3.7.4.post0), gn...",randymcmillan
4,0x2nac0nda,0.1,2021-10-06T14:34:56,None,Abdalla Abdelrhman
...,...,...,...,...,...
169616,zzy-AxCat,0.1,2022-04-28T01:59:28,None,zzy
169617,zzz001,0.0.4,2022-04-28T01:59:17,[PyYAML],Jasper Jin
169618,zzzPyPiTest,0.0.2,2022-04-26T10:14:53,None,zhaozizhe
169619,zzzPyPiTest2,0.0.2,2022-04-28T02:02:26,None,zhaozizhe


In [8]:
sorted_df = sorted_df.explode('dependency').reset_index(drop=True)
sorted_df

,name,version,upload_time,dependency,author
0,024travis-test024,0.1.0,2020-12-23T22:08:32,None,Travis Torline
1,0x-order-utils,4.0.1,2021-02-21T15:59:48,0x-contract-addresses,F. Eugene Aumson
2,0x-order-utils,4.0.1,2021-02-21T15:59:48,0x-contract-artifacts,F. Eugene Aumson
3,0x-order-utils,4.0.1,2021-02-21T15:59:48,0x-json-schemas,F. Eugene Aumson
4,0x-order-utils,4.0.1,2021-02-21T15:59:48,deprecated,F. Eugene Aumson
...,...,...,...,...,...
660143,zzy-AxCat,0.1,2022-04-28T01:59:28,None,zzy
660144,zzz001,0.0.4,2022-04-28T01:59:17,PyYAML,Jasper Jin
660145,zzzPyPiTest,0.0.2,2022-04-26T10:14:53,None,zhaozizhe
660146,zzzPyPiTest2,0.0.2,2022-04-28T02:02:26,None,zhaozizhe


### Extracting information from the dependency string

In [9]:
dependency_version_series = sorted_df['dependency'].apply(version_extractor)
dependency_name_series = sorted_df['dependency'].apply(name_extractor)

# Used just for visual purposes
dependencies_df = pd.concat([dependency_name_series, dependency_version_series], axis=1, ignore_index=True)
dependencies_df.columns = ['dependency_name', 'dependency_version']
dependencies_df

,dependency_name,dependency_version
0,None,>=0.0.0
1,0x-contract-addresses,>=0.0.0
2,0x-contract-artifacts,>=0.0.0
3,0x-json-schemas,>=0.0.0
4,deprecated,>=0.0.0
...,...,...
660143,None,>=0.0.0
660144,PyYAML,>=0.0.0
660145,None,>=0.0.0
660146,None,>=0.0.0


In [10]:
sorted_df['dependency'] = dependency_name_series
sorted_df.insert(4, 'dependency_version', dependency_version_series)

In [14]:
def convert_to_normalized_format(grouped_df: DataFrame):
    # print(grouped_df)
    normalized_form = {
        # We know the name is the same for all rows
        'name': grouped_df['name'].iloc[0],
        'versions': {}
    }
    for index, version in enumerate(grouped_df['version']):
        normalized_form['versions'][version] = {
            'timestamp': grouped_df['upload_time'].iloc[index],
            'dependencies': {}
        }
        for dependency, dependency_version in zip(grouped_df['dependency'], grouped_df['dependency_version']):
            # Some packages might have no dependencies
            if dependency is not None:
                normalized_form['versions'][version]['dependencies'][dependency] = dependency_version

    return normalized_form

# Only drop entries if they are missing required info for name, version and upload_time
normalized_df: DataFrame = sorted_df.copy().dropna(subset=['name', 'version', 'upload_time'])
normalized_json_df = normalized_df.groupby('name').apply(convert_to_normalized_format)
normalized_json_df

name
024travis-test024    {'name': '024travis-test024', 'versions': {'0....
0x-order-utils       {'name': '0x-order-utils', 'versions': {'4.0.1...
0x-python            {'name': '0x-python', 'versions': {'1.0.16': {...
0x20bf               {'name': '0x20bf', 'versions': {'0.0.1': {'tim...
0x2nac0nda           {'name': '0x2nac0nda', 'versions': {'0.1': {'t...
                                           ...                        
zzy-AxCat            {'name': 'zzy-AxCat', 'versions': {'0.1': {'ti...
zzz001               {'name': 'zzz001', 'versions': {'0.0.4': {'tim...
zzzPyPiTest          {'name': 'zzzPyPiTest', 'versions': {'0.0.2': ...
zzzPyPiTest2         {'name': 'zzzPyPiTest2', 'versions': {'0.0.2':...
zzzeeksphinx         {'name': 'zzzeeksphinx', 'versions': {'1.3.3':...
Length: 169165, dtype: object

### Saving the processed data to file

In [15]:
normalized_df.to_csv('../../data/output/pypi-repology-dependencies.csv', index=False)
normalized_json_df.to_json('../../data/output/pypi-repology-dependencies.json', orient='records')

## Converting Releases JSON to a DataFrame
Decided against using releases as they do not contain meaningful information. Most of the time they only contain the most recent version that can be recovered from the info JSON

In [ ]:
# releases_df = pd.DataFrame(pypi_data['releases'].values.tolist()) --- Runs out of memory
# releases_df = pypi_data['releases'].map(lambda x: x.keys())

### Bits and bobs that were tinkered with but were scrapped

In [ ]:
# test = pypi_data['info'].apply(lambda el: json.loads(json.dumps(el)))
# test
# pypi_data['info'][0]


In [ ]:
# pypi_data['info'].to_json('../../data/repology/pypi_info.json', orient='records', lines=True)

In [ ]:
# pypi_data_reduced = pd.read_json('../../data/repology/pypi_info.json', orient='records', lines=True)

In [ ]:
# pypi_data_reduced.dropna(subset=['requires_dist'], inplace=True)
# pypi_data_reduced.reset_index(drop=True, inplace=True)
# # pypi_data_reduced['requires_dist'] = pypi_data_reduced['requires_dist'].apply(json.loads)
# pypi_data_reduced['requires_dist'] = [','.join(x) for x in pypi_data_reduced['requires_dist']]
# pypi_data_reduced[['name', 'requires_dist']]
# Select from pypi_data_reduced all the data that has name zzzzls-Spider
# pypi_data_reduced[pypi_data_reduced['name'] == 'pandas']['requires_dist'].values[0]
# pypi_data_reduced['']

In [ ]:
# with open('../../data/repology/pypicache.json', 'r') as file:
#     json_data = json.load(file)

In [ ]:
# nested_json_data = pd.json_normalize(json_data, max_level=2)
# nested_json_data

In [ ]:
# chunks = pd.read_json('../../data/repology/pypicache.json', lines=True, chunksize=100000)
#
# for chunk in chunks:
#     display(chunk)

In [ ]:
# Initial method for converting to a normalized format. Was hard to read, and it contained quite a few bugs. Decided it was better to rewrite it
# def convert_to_normalized_format(grouped_df: DataFrameGroupBy):
#     return_list = []
#     for _, rows in grouped_df:
#         inner_dict = {
#             'name': rows['name'].values[0],
#             'versions': {
#                 rows['version'].values[0]: {
#                     'timestamp': rows['upload_time'].values[0],
#                     'dependencies': {}
#                 }
#             }
#         }
#         for dep, v in zip(rows['dependency'].values, rows['dependency_version'].values):
#             inner_dict['versions'][rows['version'].values[0]]['dependencies'] |= {dep: v}
#         return_list.append(inner_dict)
#     return pd.DataFrame(return_list)

In [ ]:
# normalized_df: DataFrame = info_df.copy().dropna()
# display(normalized_df.loc[normalized_df['name'] == '024travis-test024'])
# normalized_json_df = normalized_df.groupby('name').pipe(convert_to_normalized_format)
# normalized_json_df

In [ ]:
# Attempt to use multiprocessing. Ended up not using it since I discovered that using built-in strings is extremely fast compared to regex.
# cores = multiprocessing.cpu_count()
# chunks = np.array_split(info_df['dependency'], cores)
#
# with Pool(cores) as pool:
#     processed = pd.concat(pool.map(extract_semantic_version, chunks), ignore_index=True)

# processed
# info_df.dependency.str.extract(compiled_rx)